In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [5]:
# Define the base URL and parameters for the job search
base_url="https://www.shine.com/job-search/software-jobs"
params={
    'q':'software',
    'qActual':'software',
    'location':[242,309,331,292,383,288,365,406,313,453,249,244,246,350,391,285,247,357,243,458,323,361,522],
    'fexp':[1,2,3],
    'fsalary':[0,1,2,3,4,5,6],
    'sort':1
}



In [15]:
# Set headers to mimic a browser visit and avoid blocking
headers = {
    "User-Agent": "Mozilla/5.0"
}

response=requests.get(base_url,params=params,headers=headers)
print(response.status_code) #to make sure we are getting 200 OK response


200


In [8]:
soup=BeautifulSoup(response.content,'html.parser')

In [98]:
job_card=soup.find_all('div',class_='jobCardNova_bigCard__W2xn3 jdbigCard')

In [123]:
def extract_job_info(card):
    job={}

    #title
    title_tag=card.find('h3',class_='jobCardNova_bigCardTopTitleHeading__Rj2sC jdTruncation')
    job['Title']=title_tag.find('a').text.strip() if title_tag else None

    #Employer name
    employer_tag=card.find('span',class_='jobCardNova_bigCardTopTitleName__M_W_m jdTruncationCompany')
    job['Employer Name']=employer_tag.text.strip() if employer_tag else None

    #Experience and Salary
    exp_package_tags = card.find_all("span", class_="jobCardNova_bigCardCenterListExp__KTSEc")
    job['Experience']=exp_package_tags[0].text.strip() if len(exp_package_tags) > 0 else None
    job['Salary']=exp_package_tags[1].text.strip() if len(exp_package_tags) > 1 else None

     # Location
    location_tag = card.find("div", class_="jobCardNova_bigCardCenterListLoc__usiPB jobCardNova_limits__G87pQ d-flex justify-content-start align-items-center")
    values= location_tag.text.strip() if location_tag else None
    #few jobs have multiple location lets just choose one
    value=values.split(',')[0] if values else None
    job['Location'] = value

    #skills required
    skills_tag=card.find_all('div',class_='jobCardNova_bigCardBottomSkills__m3F1d d-flex justify-content-start align-items-center jdSkills')
    skill=skills_tag[0].text.split(',') if skills_tag else None
    job['Skills Required']=skill

    #Posted Date
    posted_tag=card.find('span',class_='jobCardNova_postedData__LTERc')
    job['Posted Date']=posted_tag.text.strip() if posted_tag else None

    return job

In [124]:
# Iterate through job_card and extract skills for each card
skills_tags = job_card[1].find_all('div', class_="jobCardNova_bigCardBottomSkills__m3F1d d-flex justify-content-start align-items-center jdSkills")
skills_tags[0].text


'business support, database, intelligence, mis, office, problem, service, technical, desk, management, analysis, data,thinking, analyst, clerk, reasoning, solving, analytic, critical, engineer, executives'

In [128]:
# Loop through all job cards and extract info
job_data = []
for card in job_card:
    job = extract_job_info(card)
    job_data.append(job)

# Preview the first few jobs
for job in job_data:
    print(job)

{'Title': 'Management Accountant', 'Employer Name': 'Garima Interprises', 'Experience': '3 to 8 Yrs', 'Salary': '2.0  - 9 LPA', 'Location': 'Kolkata', 'Skills Required': ['budget billing', ' forensic', ' senior', ' finance', ' corporate', 'administrator', ' clerk', ' invoice', ' auditor', ' financial', ' accountant', ' internal', ' controller', ' associate', ' assistant', ' analyst', ' specialist', ' admin', ' manager'], 'Posted Date': 'posted 0 days ago'}
{'Title': 'Mis Executive', 'Employer Name': 'Garima Interprises', 'Experience': '5 to 10 Yrs', 'Salary': '2.0  - 12 LPA', 'Location': 'Kolkata', 'Skills Required': ['business support', ' database', ' intelligence', ' mis', ' office', ' problem', ' service', ' technical', ' desk', ' management', ' analysis', ' data', 'thinking', ' analyst', ' clerk', ' reasoning', ' solving', ' analytic', ' critical', ' engineer', ' executives'], 'Posted Date': 'posted 0 days ago'}
{'Title': 'UI/UX Design Intern', 'Employer Name': 'Beryl Agency', 'Exp

In [135]:
#scrapping from diffrent pages
for page in range(427,642):
    if(page==1):
        url = "https://www.shine.com/job-search/software-jobs"
    else:
        url = f"https://www.shine.com/job-search/software-jobs-{page}"
    try:
        response = requests.get(url, params=params, headers=headers,timeout=10)
        soup = BeautifulSoup(response.text, "html.parser")
        job_cards = soup.find_all("div", class_="jobCardNova_bigCard__W2xn3 jdbigCard")

        #extract and convert into df
        page_data=[extract_job_info(card) for card in job_cards]
        df_page=pd.DataFrame(page_data)

        #saving to csv
        df_page.to_csv('data/job_listings.csv',mode='a',index=False,header=not pd.io.common.file_exists('job_listings.csv'))
        print(f"Saved page {page} with number of postings {len(page_data)}")
        time.sleep(2)  # Pause to give time for server to load
    except Exception as e:
        print(f"Failed to process page {page}: {e}")
        continue

Saved page 427 with number of postings 20
Saved page 428 with number of postings 20
Saved page 429 with number of postings 20
Saved page 430 with number of postings 20
Saved page 431 with number of postings 20
Saved page 432 with number of postings 20
Saved page 433 with number of postings 20
Saved page 434 with number of postings 20
Saved page 435 with number of postings 20
Saved page 436 with number of postings 20
Saved page 437 with number of postings 20
Saved page 438 with number of postings 20
Saved page 439 with number of postings 20
Saved page 440 with number of postings 20
Saved page 441 with number of postings 20
Saved page 442 with number of postings 20
Saved page 443 with number of postings 20
Saved page 444 with number of postings 20
Saved page 445 with number of postings 20
Saved page 446 with number of postings 20
Saved page 447 with number of postings 20
Saved page 448 with number of postings 20
Saved page 449 with number of postings 20
Saved page 450 with number of post

In [133]:
len(job_data)

120

In [136]:
df=pd.read_csv('job_listings.csv')
df.sample(5)

,Title,Employer Name,Experience,Salary,Location,Skills Required,Posted Date
209,Process Excellence - Senior Analyst (Lean Six ...,Asian Hires,2 to 6 Yrs,NaN,Vadodara,"['Data Analysis Root Cause Analysis', ' Six Si...",posted 1 day ago
6663,PR Manager,Hot Fuego LLC,5 to 9 Yrs,NaN,Pune,"['Media Relations Press Outreach', 'Brand Repu...",posted 4 weeks ago
9095,Medical Assistant,American Family Care,0 to 3 Yrs,NaN,Rajasthan,"['Patient Care Medical History', ' Sterilizati...",posted 2 months ago
1648,Educational Counsellor,Capital Placement Services,2 to 8 Yrs,NaN,Haryana,"['Calling Telesales', ' SAAS', ' VOC', ' B2B',...",posted 6 days ago
9412,Associate Lecturer / Lecturer in Electrical En...,UNSW,0 to 4 Yrs,NaN,Delhi,"['Electrical Engineering Energy Systems', ' Em...",posted 2 months ago


In [143]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12560 entries, 0 to 12559
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Title            12560 non-null  object
 1   Employer Name    12560 non-null  object
 2   Experience       12560 non-null  object
 3   Salary           680 non-null    object
 4   Location         12560 non-null  object
 5   Skills Required  12560 non-null  object
 6   Posted Date      12560 non-null  object
dtypes: object(7)
memory usage: 687.0+ KB
